In [8]:
import pandas as pd
import numpy as np
import os

def generar_tabla_top10_ranking():
    """
    Genera tabla LaTeX con los top 10 mejores resultados por accuracy
    """
    df = pd.read_csv("results/evaluation_results/evaluation_results.csv")
    top10 = df.nlargest(10, 'accuracy')
    
    latex_code = """\\begin{table}[htbp]
\\centering
\\caption{Top 10 Mejores Resultados por Accuracy}
\\label{tab:top10_results}
\\small
\\begin{tabular}{clllcc}
\\toprule
\\textbf{Rank} & \\textbf{Algoritmo} & \\textbf{Encoding} & \\textbf{Tipo} & \\textbf{Accuracy} & \\textbf{F1-Score} \\\\
\\midrule
"""
    
    for i, (_, row) in enumerate(top10.iterrows(), 1):
        algoritmo = row['algorithm']
        encoding_clean = row['encoding'].replace('_', '\\_')
        tipo = "AS" if row['encoding'].startswith('AS_') else "PS"
        
        latex_code += f"{i} & {algoritmo} & {encoding_clean} & {tipo} & "
        latex_code += f"{row['accuracy']:.3f} & {row['f1_score_weighted']:.3f} \\\\\\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\\\[0.5em]
\\footnotesize
\\textit{AS: Secuencias Alineadas; PS: Secuencias con Padding}
\\end{table}"""
    
    filename = "latex_tables/tabla_top10_ranking.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla top 10 guardada en: {filename}")

def generar_tabla_randomforest_completa():
    """
    Genera tabla completa para Random Forest con métricas de entrenamiento y evaluación
    """
    df_train = pd.read_csv("results/training_results/resultados_entrenamiento_resumen.csv")
    df_eval = pd.read_csv("results/evaluation_results/evaluation_results.csv")
    
    df_train['model_name']=df_train['model_name'].replace({'RandomForest':'Random Forest'})
    df_eval['algorithm']=df_eval['algorithm'].replace({'RandomForest':'Random Forest'})
    
    df_train_rf = df_train[df_train['model_name'] == 'Random Forest'].copy()
    df_eval_rf = df_eval[df_eval['algorithm'] == 'Random Forest'].copy()
    
    df_combined = pd.merge(
        df_train_rf[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        df_eval_rf[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        on='encoding', suffixes=('_train', '_eval')
    )
    
    df_combined = df_combined.sort_values('accuracy_eval', ascending=False)
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Todas las Métricas - Random Forest}
\\label{tab:random_forest_complete}
\\scriptsize
\\begin{tabular}{lcccccccccc}
\\toprule
& \\multicolumn{2}{c}{\\textbf{Accuracy}} & \\multicolumn{2}{c}{\\textbf{F1-Score Weighted}} & \\multicolumn{2}{c}{\\textbf{F1-Score Macro}} & \\multicolumn{2}{c}{\\textbf{Precision Weighted}} & \\multicolumn{2}{c}{\\textbf{Precision Macro}} \\\\
\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7} \\cmidrule(lr){8-9} \\cmidrule(lr){10-11}
\\textbf{Encoding} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} \\\\
\\midrule
"""
    
    for _, row in df_combined.iterrows():
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{encoding} & "
        latex_code += f"{row['accuracy_train']:.3f} & \\textbf{{{row['accuracy_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_weighted_train']:.3f} & \\textbf{{{row['f1_score_weighted_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_macro_train']:.3f} & \\textbf{{{row['f1_score_macro_eval']:.3f}}} & "
        latex_code += f"{row['precision_weighted_train']:.3f} & \\textbf{{{row['precision_weighted_eval']:.3f}}} & "
        latex_code += f"{row['precision_macro_train']:.3f} & \\textbf{{{row['precision_macro_eval']:.3f}}} \\\\\\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table*}"""
    
    filename = "latex_tables/tabla_randomforest_completa.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla Random Forest completa guardada en: {filename}")

def generar_tabla_svm_completa():
    """
    Genera tabla completa para SVM con métricas de entrenamiento y evaluación
    """
    df_train = pd.read_csv("results/training_results/resultados_entrenamiento_resumen.csv")
    df_eval = pd.read_csv("results/evaluation_results/evaluation_results.csv")
    
    df_train_svm = df_train[df_train['model_name'] == 'SVM'].copy()
    df_eval_svm = df_eval[df_eval['algorithm'] == 'SVM'].copy()
    
    df_combined = pd.merge(
        df_train_svm[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        df_eval_svm[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        on='encoding', suffixes=('_train', '_eval')
    )
    
    df_combined = df_combined.sort_values('accuracy_eval', ascending=False)
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Todas las Métricas - SVM}
\\label{tab:svm_complete}
\\scriptsize
\\begin{tabular}{lcccccccccc}
\\toprule
& \\multicolumn{2}{c}{\\textbf{Accuracy}} & \\multicolumn{2}{c}{\\textbf{F1-Score Weighted}} & \\multicolumn{2}{c}{\\textbf{F1-Score Macro}} & \\multicolumn{2}{c}{\\textbf{Precision Weighted}} & \\multicolumn{2}{c}{\\textbf{Precision Macro}} \\\\
\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7} \\cmidrule(lr){8-9} \\cmidrule(lr){10-11}
\\textbf{Encoding} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} \\\\
\\midrule
"""
    
    for _, row in df_combined.iterrows():
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{encoding} & "
        latex_code += f"{row['accuracy_train']:.3f} & \\textbf{{{row['accuracy_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_weighted_train']:.3f} & \\textbf{{{row['f1_score_weighted_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_macro_train']:.3f} & \\textbf{{{row['f1_score_macro_eval']:.3f}}} & "
        latex_code += f"{row['precision_weighted_train']:.3f} & \\textbf{{{row['precision_weighted_eval']:.3f}}} & "
        latex_code += f"{row['precision_macro_train']:.3f} & \\textbf{{{row['precision_macro_eval']:.3f}}} \\\\\\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table*}"""
    
    filename = "latex_tables/tabla_svm_completa.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla SVM completa guardada en: {filename}")

def generar_tabla_xgboost_completa():
    """
    Genera tabla completa para XGBoost con métricas de entrenamiento y evaluación
    """
    df_train = pd.read_csv("results/training_results/resultados_entrenamiento_resumen.csv")
    df_eval = pd.read_csv("results/evaluation_results/evaluation_results.csv")
    
    df_train_xgb = df_train[df_train['model_name'] == 'XGBoost'].copy()
    df_eval_xgb = df_eval[df_eval['algorithm'] == 'XGBoost'].copy()
    
    df_combined = pd.merge(
        df_train_xgb[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        df_eval_xgb[['encoding', 'accuracy', 'f1_score_weighted', 'f1_score_macro', 'precision_weighted', 'precision_macro']],
        on='encoding', suffixes=('_train', '_eval')
    )
    
    df_combined = df_combined.sort_values('accuracy_eval', ascending=False)
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Todas las Métricas - XGBoost}
\\label{tab:xgboost_complete}
\\scriptsize
\\begin{tabular}{lcccccccccc}
\\toprule
& \\multicolumn{2}{c}{\\textbf{Accuracy}} & \\multicolumn{2}{c}{\\textbf{F1-Score Weighted}} & \\multicolumn{2}{c}{\\textbf{F1-Score Macro}} & \\multicolumn{2}{c}{\\textbf{Precision Weighted}} & \\multicolumn{2}{c}{\\textbf{Precision Macro}} \\\\
\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7} \\cmidrule(lr){8-9} \\cmidrule(lr){10-11}
\\textbf{Encoding} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} & Train & \\textbf{Eval} \\\\
\\midrule
"""
    
    for _, row in df_combined.iterrows():
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{encoding} & "
        latex_code += f"{row['accuracy_train']:.3f} & \\textbf{{{row['accuracy_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_weighted_train']:.3f} & \\textbf{{{row['f1_score_weighted_eval']:.3f}}} & "
        latex_code += f"{row['f1_score_macro_train']:.3f} & \\textbf{{{row['f1_score_macro_eval']:.3f}}} & "
        latex_code += f"{row['precision_weighted_train']:.3f} & \\textbf{{{row['precision_weighted_eval']:.3f}}} & "
        latex_code += f"{row['precision_macro_train']:.3f} & \\textbf{{{row['precision_macro_eval']:.3f}}} \\\\\\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\end{table*}"""
    
    filename = "latex_tables/tabla_xgboost_completa.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla XGBoost completa guardada en: {filename}")

def generar_tabla_encoding_estadisticas():
    """
    Genera tabla con estadísticas de tiempo y memoria por encoding
    """
    # Leer el archivo CSV con los resultados
    df = pd.read_csv("datos/resultados_codificacion_tiempo_memoria.csv")
    
    # Calcular estadísticas de tiempo por encoding
    time_stats = df.groupby('encoding').agg({
        'time': ['mean', 'std']
    }).round(4)
    
    # Aplanar los nombres de columnas
    time_stats.columns = ['avg_time', 'std_time']
    time_stats = time_stats.reset_index()
    
    # Calcular estadísticas de memoria por encoding (promedio)
    memory_stats = df.groupby('encoding').agg({
        'memory_used': 'mean'
    }).round(2).reset_index()
    
    # Combinar las estadísticas
    stats_combined = pd.merge(time_stats, memory_stats, on='encoding')
    
    # Ordenar por tiempo promedio
    stats_combined = stats_combined.sort_values('avg_time')
    
    # Crear el directorio si no existe
    os.makedirs("latex_tables", exist_ok=True)
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Estadísticas de Tiempo y Memoria por Encoding}
\\label{tab:encoding_stats}
\\scriptsize
\\begin{tabular}{lccc}
\\toprule
& \\multicolumn{2}{c}{\\textbf{Tiempo (segundos)}} & \\textbf{Memoria} \\\\
\\cmidrule(lr){2-3} \\cmidrule(lr){4-4}
\\textbf{Encoding} & Promedio & Desv. Std & (MB) \\\\
\\midrule
"""
    
    for _, row in stats_combined.iterrows():
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{encoding} & "
        latex_code += f"{row['avg_time']:.4f} & "
        latex_code += f"{row['std_time']:.4f} & "
        latex_code += f"{row['memory_used']:.2f} \\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\\\[0.5em]
\\footnotesize
\\textit{Nota: Las estadísticas se calcularon basándose en múltiples ejecuciones por encoding. El tiempo se reporta en segundos y la memoria en megabytes.}
\\end{table*}"""
    
    filename = "latex_tables/tabla_encoding_estadisticas.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla estadísticas de encoding guardada en: {filename}")

def generar_tabla_tiempos_entrenamiento():
    """
    Genera tabla con tiempos de entrenamiento por modelo y encoding
    """
    df = pd.read_csv("results/training_times/training_times_complete_detailed.csv")
    
    stats_df = df.groupby(['modelo', 'encoding']).agg({
        'tiempo': ['count', 'mean', 'std']
    }).round(2)
    
    stats_df.columns = ['num_ejecuciones', 'tiempo_promedio_seg', 'tiempo_desv_std']
    stats_df = stats_df.reset_index()
    stats_df = stats_df.sort_values('tiempo_promedio_seg')
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Tiempos de Entrenamiento por Modelo y Encoding}
\\label{tab:training_times}
\\scriptsize
\\renewcommand{\\arraystretch}{1.0}
\\begin{tabular}{llcc}
\\toprule
& & \\multicolumn{2}{c}{\\textbf{Tiempo (segundos)}} \\\\
\\cmidrule(lr){3-4}
\\textbf{Modelo} & \\textbf{Encoding} & Promedio & Desv. Std \\\\
\\midrule
"""
    
    for _, row in stats_df.iterrows():
        modelo = row['modelo']
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{modelo} & {encoding} & "
        latex_code += f"{row['tiempo_promedio_seg']:.2f} & "
        latex_code += f"{row['tiempo_desv_std']:.2f} \\\\\n"
    
    total_iteraciones = df['iter'].nunique()
    
    latex_code += f"""\\bottomrule
\\end{{tabular}}
\\\\[0.5em]
\\footnotesize
\\textit{{Nota: Cada combinación modelo-encoding fue ejecutada {total_iteraciones} veces para el análisis de tiempos de entrenamiento.}}
\\end{{table*}}"""
    
    filename = "latex_tables/tabla_tiempos_entrenamiento.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla tiempos de entrenamiento guardada en: {filename}")

def generar_tabla_tiempos_evaluacion():
    """
    Genera tabla con tiempos de evaluación por modelo y encoding
    """
    try:
        df = pd.read_excel("results/evaluation_results/evaluation_complete.xlsx", 
                          sheet_name="Tiempos_Ejecucion")
    except:
        df = pd.read_csv("results/evaluation_results.csv")
    
    df_sorted = df.sort_values('total_time' if 'total_time' in df.columns else 'prediction_time')
    
    latex_code = """\\begin{table*}[htbp]
\\centering
\\caption{Tiempos de Evaluación por Modelo y Encoding}
\\label{tab:evaluation_times}
\\scriptsize
\\renewcommand{\\arraystretch}{1.0}
\\begin{tabular}{llcccc}
\\toprule
& & \\multicolumn{4}{c}{\\textbf{Tiempo (segundos)}} \\\\
\\cmidrule(lr){3-6}
\\textbf{Modelo} & \\textbf{Encoding} & Carga & Predicción & Total & Prep. Datos \\\\
\\midrule
"""
    
    for _, row in df_sorted.iterrows():
        algoritmo = row.get('algorithm', row.get('modelo', 'N/A'))
        encoding = row['encoding'].replace('_', '\\_')
        latex_code += f"{algoritmo} & {encoding} & "
        latex_code += f"{row.get('model_load_time', 0):.3f} & "
        latex_code += f"{row.get('prediction_time', 0):.3f} & "
        latex_code += f"{row.get('total_time', 0):.3f} & "
        latex_code += f"{row.get('data_prep_time', 0):.3f} \\\\\n"
    
    latex_code += """\\bottomrule
\\end{tabular}
\\\\[0.5em]
\\footnotesize
\\textit{Nota: Tiempos medidos durante evaluación sobre 8,555 secuencias independientes.}
\\end{table*}"""
    
    filename = "latex_tables/tabla_tiempos_evaluacion.tex"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(latex_code)
    
    print(f"Tabla tiempos de evaluación guardada en: {filename}")

def generar_todas_las_tablas():
    """
    Función principal que genera todas las tablas LaTeX
    """
    os.makedirs("latex_tables", exist_ok=True)
    
    print("Generando todas las tablas LaTeX...")
    print("=" * 50)
    
    try:
        generar_tabla_top10_ranking()
        generar_tabla_randomforest_completa()
        generar_tabla_svm_completa()
        generar_tabla_xgboost_completa()
        generar_tabla_encoding_estadisticas()
        generar_tabla_tiempos_entrenamiento()
        generar_tabla_tiempos_evaluacion()
        
        print("=" * 50)
        print("TODAS LAS TABLAS GENERADAS EXITOSAMENTE")
        print("Archivos guardados en: latex_tables/")
        print("=" * 50)
        
    except Exception as e:
        print(f"Error al generar tablas: {e}")
        print("Verifique que todos los archivos de datos estén disponibles")

if __name__ == "__main__":
    generar_todas_las_tablas()

Generando todas las tablas LaTeX...
Tabla top 10 guardada en: latex_tables/tabla_top10_ranking.tex
Tabla Random Forest completa guardada en: latex_tables/tabla_randomforest_completa.tex
Tabla SVM completa guardada en: latex_tables/tabla_svm_completa.tex
Tabla XGBoost completa guardada en: latex_tables/tabla_xgboost_completa.tex
Tabla estadísticas de encoding guardada en: latex_tables/tabla_encoding_estadisticas.tex
Tabla tiempos de entrenamiento guardada en: latex_tables/tabla_tiempos_entrenamiento.tex
Tabla tiempos de evaluación guardada en: latex_tables/tabla_tiempos_evaluacion.tex
TODAS LAS TABLAS GENERADAS EXITOSAMENTE
Archivos guardados en: latex_tables/
